# Deep Search

Lets get a basic implementation of a more sophisticated search algo

### Experiments

Magic Elements

{
	"geoSupportedSlug": "Burke_VA",
	"query": {
		"unique": true,
		"status": [
			"for_sale",
			"ready_to_build"
		],
		"boundary": {
			"type": "Polygon",
			"coordinates": [
				[
					[
						-77.44898,
						38.86813
					],
					[
						-77.44,
						38.85764
					],
					[
						-77.41416,
						38.84014
					],
					[
						-77.37822,
						38.83139
					],
					[
						-77.35351,
						38.82002
					],
					[
						-77.31981,
						38.78238
					],
					[
						-77.27264,
						38.7491
					],
					[
						-77.24119,
						38.7342
					],
					[
						-77.19851,
						38.71931
					],
					[
						-77.1884,
						38.72194
					],
					[
						-77.18503,
						38.72719
					],
					[
						-77.1693,
						38.785
					],
					[
						-77.15583,
						38.80076
					],
					[
						-77.11764,
						38.80339
					],
					[
						-77.08731,
						38.79901
					],
					[
						-77.080164,
						38.79641029792285
					],
					[
						-77.080164,
						38.905333636363636
					],
					[
						-77.08169,
						38.90573
					],
					[
						-77.10079,
						38.91534
					],
					[
						-77.11178650548938,
						38.92637
					],
					[
						-77.42490248618785,
						38.92637
					],
					[
						-77.42539,
						38.91534
					],
					[
						-77.44898,
						38.87425
					],
					[
						-77.44898,
						38.86813
					]
				]
			]
		},
		"type": [
			"single_family"
		],
		"list_price": {
			"max": 700000
		}
	},
	"client_data": {
		"device_data": {
			"device_type": "desktop"
		}
	},
	"limit": 200,
	"offset": 0,
	"sort_type": "relevant",
	"search_promotion": {
		"name": "CITY",
		"slots": [
			5,
			6,
			7,
			8
		],
		"promoted_properties": [
			[]
		]
	}
}

### Basic Setup

In [28]:
import pandas as pd
import requests
import concurrent.futures
import json
from datetime import datetime

### Running Code

In [29]:
URL = "https://www.realtor.com/api/v1/hulk_main_srp"

QUERYSTRING = {"client_id": "rdc-x", "schema": "vesta"}

HEADERS = {
    "authority": "www.realtor.com",
    "accept": "application/json",
    "accept-language": "en-US,en;q=0.5",
    "content-type": "application/json",
    "origin": "https://www.realtor.com",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sec-gpc": "1",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

QUERY = """
    query ConsumerSearchMainQuery(
        $query: HomeSearchCriteria!
        $limit: Int
        $offset: Int
        $sort: [SearchAPISort]
        $sort_type: SearchSortType
        $client_data: JSON
        $bucket: SearchAPIBucket
        ) {
        home_search: home_search(
            query: $query
            sort: $sort
            limit: $limit
            offset: $offset
            sort_type: $sort_type
            client_data: $client_data
            bucket: $bucket
        ) {
            count
            total
            results {
            property_id
            list_price
            primary
            rent_to_own {
                rent
                right_to_purchase
                provider
            }
            primary_photo(https: true) {
                href
            }
            source {
                id
                agents {
                office_name
                }
                type
                spec_id
                plan_id
            }
            community {
                property_id
                permalink
                description {
                name
                }
                advertisers {
                office {
                    hours
                    phones {
                    type
                    number
                    }
                }
                builder {
                    fulfillment_id
                }
                }
            }
            products {
                brand_name
                products
            }
            listing_id
            matterport
            virtual_tours {
                href
                type
            }
            status
            permalink
            price_reduced_amount
            other_listings {
                rdc {
                listing_id
                status
                listing_key
                primary
                }
            }
            description {
                beds
                baths
                baths_full
                baths_half
                baths_1qtr
                baths_3qtr
                garage
                stories
                type
                sub_type
                lot_sqft
                sqft
                year_built
                sold_price
                sold_date
                name
            }
            location {
                street_view_url
                address {
                line
                postal_code
                state
                state_code
                city
                coordinate {
                    lat
                    lon
                }
                }
                county {
                name
                fips_code
                }
            }
            tax_record {
                public_record_id
            }
            lead_attributes {
                show_contact_an_agent
                opcity_lead_attributes {
                cashback_enabled
                flip_the_market_enabled
                }
                lead_type
                ready_connect_mortgage {
                show_contact_a_lender
                show_veterans_united
                }
            }
            open_houses {
                start_date
                end_date
                description
                methods
                time_zone
                dst
            }
            flags {
                is_coming_soon
                is_pending
                is_foreclosure
                is_contingent
                is_new_construction
                is_new_listing(days: 14)
                is_price_reduced(days: 30)
                is_plan
                is_subdivision
            }
            list_date
            last_update_date
            coming_soon_date
            photos(limit: 2, https: true) {
                href
            }
            tags
            branding {
                type
                photo
                name
            }
            }
        }
    }
"""

Lets build a super complicated query

In [30]:
Boundary = {
			"type": "Polygon",
			"coordinates": [
				[
					[
						-77.44898,
						38.86813
					],
					[
						-77.44,
						38.85764
					],
					[
						-77.41416,
						38.84014
					],
					[
						-77.37822,
						38.83139
					],
					[
						-77.35351,
						38.82002
					],
					[
						-77.31981,
						38.78238
					],
					[
						-77.27264,
						38.7491
					],
					[
						-77.24119,
						38.7342
					],
					[
						-77.19851,
						38.71931
					],
					[
						-77.1884,
						38.72194
					],
					[
						-77.18503,
						38.72719
					],
					[
						-77.1693,
						38.785
					],
					[
						-77.15583,
						38.80076
					],
					[
						-77.11764,
						38.80339
					],
					[
						-77.08731,
						38.79901
					],
					[
						-77.080164,
						38.79641029792285
					],
					[
						-77.080164,
						38.905333636363636
					],
					[
						-77.08169,
						38.90573
					],
					[
						-77.10079,
						38.91534
					],
					[
						-77.11178650548938,
						38.92637
					],
					[
						-77.42490248618785,
						38.92637
					],
					[
						-77.42539,
						38.91534
					],
					[
						-77.44898,
						38.87425
					],
					[
						-77.44898,
						38.86813
					]
				]
			]
		}

In [31]:
VQUERY = {
    "status": ["for_sale", "ready_to_build"],
    "primary": True,
    "type": ["single_family"],
	"list_price": {"max": 700000},
    # "search_location": {"location": "Burke, VA"}
    "boundary": Boundary
}

In [32]:
VQUERY

{'status': ['for_sale', 'ready_to_build'],
 'primary': True,
 'type': ['single_family'],
 'list_price': {'max': 700000},
 'boundary': {'type': 'Polygon',
  'coordinates': [[[-77.44898, 38.86813],
    [-77.44, 38.85764],
    [-77.41416, 38.84014],
    [-77.37822, 38.83139],
    [-77.35351, 38.82002],
    [-77.31981, 38.78238],
    [-77.27264, 38.7491],
    [-77.24119, 38.7342],
    [-77.19851, 38.71931],
    [-77.1884, 38.72194],
    [-77.18503, 38.72719],
    [-77.1693, 38.785],
    [-77.15583, 38.80076],
    [-77.11764, 38.80339],
    [-77.08731, 38.79901],
    [-77.080164, 38.79641029792285],
    [-77.080164, 38.905333636363636],
    [-77.08169, 38.90573],
    [-77.10079, 38.91534],
    [-77.11178650548938, 38.92637],
    [-77.42490248618785, 38.92637],
    [-77.42539, 38.91534],
    [-77.44898, 38.87425],
    [-77.44898, 38.86813]]]}}

In [33]:
results = []

offset = 0
# total gets updated, just need a value greater than offset for first iteration
total = offset + 1

# for the purpose of this example, creating a max
max_results = 200


In [34]:
VARIABLES = {
            "query": VQUERY,
            "limit": 42,
            "offset": offset,
            "sort_type": "relevant",
            "by_prop_type": ["home"]
        }

In [35]:
VARIABLES

{'query': {'status': ['for_sale', 'ready_to_build'],
  'primary': True,
  'type': ['single_family'],
  'list_price': {'max': 700000},
  'boundary': {'type': 'Polygon',
   'coordinates': [[[-77.44898, 38.86813],
     [-77.44, 38.85764],
     [-77.41416, 38.84014],
     [-77.37822, 38.83139],
     [-77.35351, 38.82002],
     [-77.31981, 38.78238],
     [-77.27264, 38.7491],
     [-77.24119, 38.7342],
     [-77.19851, 38.71931],
     [-77.1884, 38.72194],
     [-77.18503, 38.72719],
     [-77.1693, 38.785],
     [-77.15583, 38.80076],
     [-77.11764, 38.80339],
     [-77.08731, 38.79901],
     [-77.080164, 38.79641029792285],
     [-77.080164, 38.905333636363636],
     [-77.08169, 38.90573],
     [-77.10079, 38.91534],
     [-77.11178650548938, 38.92637],
     [-77.42490248618785, 38.92637],
     [-77.42539, 38.91534],
     [-77.44898, 38.87425],
     [-77.44898, 38.86813]]]}},
 'limit': 42,
 'offset': 0,
 'sort_type': 'relevant',
 'by_prop_type': ['home']}

In [36]:

while offset < total:

    print(f"handling offset {offset} in a total of {total}      ", end='\r')

    payload = {
        "query": QUERY,
       "variables": VARIABLES,
        "operationName": "ConsumerSearchMainQuery",
        "callfrom": "SRP",
        "nrQueryType": "MAIN_SRP",
        "isClient": True,
    }

    response = requests.request(
        "POST", URL, json=payload, headers=HEADERS, params=QUERYSTRING)

    if response.status_code != 200:
        raise ValueError(f"Bad status code on response: {response.status_code}")

    try:
        data = response.json()['data']['home_search']
    except:
        print("Failed to read data, something went wrong with the request")
        raise

    total = data['total']

    response_results = data['results']
    offset += len(response_results)

    results += response_results

    if max_results and offset >= max_results:
        print("\n")
        print(f"Hit max: {max_results}")
        break

print("Done!")

Done!ing offset 42 in a total of 56      


In [ ]:
type(results)

list

In [ ]:
len(results)

84